In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.datasets import load_iris
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.metrics import classification_report_imbalanced

df = pd.read_csv('cleaned_data_label.csv', sep=',')
#lineList = [line.rstrip('\n') for line in open('StopWord.txt',encoding="utf-8")]
#lineList.sort()
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
#df['comment_content'] = df['comment_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (lineList)]))

#target = ['Chat Luong San Pham','Dong goi','Giao hang','Cham soc khach hang','Thiet ke','Gia Ban','Pin','Do Nhay','Phu Kien','Nut Power','Con Lan','Ket Noi','Phi Ship']

target = ['Chat Luong San Pham','Dong goi','Giao hang','Cham soc khach hang','Thiet ke','Gia Ban']

for i in range(len(target)):
    X_train, X_test, y_train, y_test = train_test_split(df['comment_content'], df[target[i]], test_size=0.25,
                                                        stratify=df[target[i]], random_state=123456)
    vectorizer = TfidfVectorizer(min_df=20)
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    nbg = GaussianNB()
    nbg.fit(vectors_train.toarray(), y_train)
    predicted = nbg.predict(vectors_test.toarray())
    accuracy = metrics.f1_score(y_test, predicted)
    print(target[i],': ',accuracy)
    #print(classification_report_imbalanced(y_test, predicted))

Chat Luong San Pham :  0.8071748878923767
Dong goi :  0.8367346938775511
Giao hang :  0.5581395348837209
Cham soc khach hang :  0.6470588235294118
Thiet ke :  0.12698412698412698
Gia Ban :  0.11320754716981132
